# LangChain: Agents

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

Sometimes people think of a large language model 
as a knowledge store, as if it's learned to memorize a 
lot of information, maybe off the internet, so 
when you ask it a question, it can answer the question. 
But I think a even more useful way to think of a 
large language model is sometimes as a reasoning engine, 
in which you can give it chunks of text or other sources 
of information. 
And then the large language model, LLM, will maybe use 
this background knowledge that's learned off the internet, but 
to use the new information you give it 
to help you answer questions or reason through 
content or decide even what to do next. And that's what 
LangChain's Agents framework helps you to do. 
 
Agents are probably my favorite part of LangChain. 
I think they're also one of the most powerful parts, but 
they're also one of the newer parts. So we're 
seeing a lot of stuff emerge here that's really new to 
everyone in the field. And so this should be a very exciting lesson 
as we dive into what agents are, how to create and 
how to use agents, how to equip them with different types of 
tools, like search engines that come built into LangChain, 
and then also how to create your own 
tools, so that you can let agents interact with any 
data stores, any APIs, any functions that you 
might want them to. 

## Enviroment

In [ ]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings("ignore")

## Built-in LangChain tools

In [ ]:
#!pip install -U wikipedia

In [ ]:
# installing packages
from langchain.agents.agent_toolkits import create_python_agent
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.chat_models import ChatOpenAI

We're going to initialize a language model. We're going to use Chat OpenAI, and importantly, we're going to set the temperature equal to zero. This is important because we're going to be using the language model as the reasoning.

Engine of an agent where it's connecting to other sources of data and computation. And so we want this reasoning engine to be as good and as precise as possible. And so we're going to set it to zero to get rid of any randomness that might arise.

In [ ]:
llm = ChatOpenAI(temperature=0)

The two tools that we're going to load are the LLM math tool and the Wikipedia tool. The LLM math tool is actually a chain.

Itself, which uses a language model in conjunction with a calculator to do math problems. The Wikipedia tool is an API that connects to Wikipedia, allowing you to run search queries against Wikipedia and get back results.

In [ ]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

We're going to initialize the agent with the tools, the language model and then an agent type. Here we're going to use chat zero shot react description. The important things to note here are first chat this is an agent that has been optimized to work with chat models and second react.

This is a prompting technique designed to get the best reasoning performance out of language models. We're also going to pass in handle parsing errors equals true. This is useful when the language model might output something that is not able to be parsed into an action and action input which is the desired output.

When this happens we'll actually pass the misformatted text back to the language model and ask it to correct itself. And finally we're going to pass in verbose equals true. This is going to print out a bunch of steps that makes it really clear to us in the jupyter notebook what's going on.

In [ ]:
# Initialize an agent with tools, llm and an agent type
agent= initialize_agent(
    tools, 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [ ]:
agent("What is the 25% of 300?")

*OUTPUT*

![Agent](immagini/29_agent.png)

So we can see here that when it enters the agent executor chain that it first thinks about what it needs to do.

So it has a thought, it then has an action. And this action is actually a JSON Blob, corresponding to two things an action and an action input. The action corresponds to the tool to use.

So here it says Calculator. The action input is the input to that tool and here it's a string of 300 times 00.25. Next we can see that there's observation with answer in a separate color.

This observation answer equals 75.0 is actually coming from the calculator tool itself. Next we go back to the language model.

When the text turns to green, we have the answer to the question final answer 75.0 and that's the output that we get.

In [ ]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question) 

*OUTPUT*

![Agent](immagini/30_agent.png)

The observation that comes back from Wikipedia is actually two results, two pages.

As there's two different Tom Mitchells. We can see the first one covers the computer scientist and the second one it looks like it's an Australian footballer.

## Python Agent

If you've seen things like Copilot or even Chat GPT with the Code interpreter plugin enabled, one of the things they're doing is they're using the language model to write code and then executing that code.

We're going to create a Python agent and we're going to use the same LLM as before, and we're going to give it a tool, the Python REPL Tool. A REPL is basically a way to interact with code.

You can think of it as a jupyter notebook. So the agent can execute code with this REPL. It will then run, and then we'll get back some results, and those results will be passed back into the agent so it can decide what to do next.

In [ ]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [ ]:
# The problem that we're going to have this agent solve is 
# we're going to give it a list of names and then ask it to sort them.

customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [ ]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 

*OUTPUT*

![Agent](immagini/31_agent.png)

We're asking it to print the output so that it can actually see what the result is. These printed statements are what's going to be fed back into the language model later on, so it can reason about the output of the code that it just ran.

#### View detailed outputs of the chains

Let's dig a little bit deeper and run this with Langchain debug set to true.

In [ ]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 
langchain.debug=False

*OUTPUT*
```python
[chain/start] [1:chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:chain:AgentExecutor > 2:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, debug your code and try again.\nOnly use the output of your code to answer the question. \nYou might know the answer without running any code, but you should still run the code to get the answer.\nIf it does not seem like you can write code to answer the question, just return \"I don't know\" as the answer.\n\n\nPython REPL: A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Python REPL]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]\nThought:"
  ]
}
[llm/end] [1:chain:AgentExecutor > 2:chain:LLMChain > 3:llm:ChatOpenAI] [3.82s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "I can use the `sorted()` function to sort the list of customers. I will need to provide a key function that specifies the sorting criteria.\nAction: Python REPL\nAction Input: `customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]`",
        "generation_info": null,
        "message": {
          "content": "I can use the `sorted()` function to sort the list of customers. I will need to provide a key function that specifies the sorting criteria.\nAction: Python REPL\nAction Input: `customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]`",
          "additional_kwargs": {},
          "example": false
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 326,
      "completion_tokens": 92,
      "total_tokens": 418
    },
    "model_name": "gpt-3.5-turbo"
  }
}
[chain/end] [1:chain:AgentExecutor > 2:chain:LLMChain] [3.82s] Exiting Chain run with output:
{
  "text": "I can use the `sorted()` function to sort the list of customers. I will need to provide a key function that specifies the sorting criteria.\nAction: Python REPL\nAction Input: `customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]`"
}
[tool/start] [1:chain:AgentExecutor > 4:tool:Python REPL] Entering Tool run with input:
"`customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]`"
[tool/end] [1:chain:AgentExecutor > 4:tool:Python REPL] [0.232ms] Exiting Tool run with output:
""
[chain/start] [1:chain:AgentExecutor > 5:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "I can use the `sorted()` function to sort the list of customers. I will need to provide a key function that specifies the sorting criteria.\nAction: Python REPL\nAction Input: `customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]`\nObservation: \nThought:",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 5:chain:LLMChain > 6:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, debug your code and try again.\nOnly use the output of your code to answer the question. \nYou might know the answer without running any code, but you should still run the code to get the answer.\nIf it does not seem like you can write code to answer the question, just return \"I don't know\" as the answer.\n\n\nPython REPL: A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Python REPL]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]\nThought:I can use the `sorted()` function to sort the list of customers. I will need to provide a key function that specifies the sorting criteria.\nAction: Python REPL\nAction Input: `customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]`\nObservation: \nThought:"
  ]
}
[llm/end] [1:chain:AgentExecutor > 5:chain:LLMChain > 6:llm:ChatOpenAI] [2.22s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "I will use the `sorted()` function to sort the list of customers. I will provide a lambda function as the key argument to specify the sorting criteria.\nAction: Python REPL\nAction Input: `sorted(customers, key=lambda x: (x[1], x[0]))`",
        "generation_info": null,
        "message": {
          "content": "I will use the `sorted()` function to sort the list of customers. I will provide a lambda function as the key argument to specify the sorting criteria.\nAction: Python REPL\nAction Input: `sorted(customers, key=lambda x: (x[1], x[0]))`",
          "additional_kwargs": {},
          "example": false
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 423,
      "completion_tokens": 58,
      "total_tokens": 481
    },
    "model_name": "gpt-3.5-turbo"
  }
}
[chain/end] [1:chain:AgentExecutor > 5:chain:LLMChain] [2.22s] Exiting Chain run with output:
{
  "text": "I will use the `sorted()` function to sort the list of customers. I will provide a lambda function as the key argument to specify the sorting criteria.\nAction: Python REPL\nAction Input: `sorted(customers, key=lambda x: (x[1], x[0]))`"
}
[tool/start] [1:chain:AgentExecutor > 7:tool:Python REPL] Entering Tool run with input:
"`sorted(customers, key=lambda x: (x[1], x[0]))`"
[tool/end] [1:chain:AgentExecutor > 7:tool:Python REPL] [0.219ms] Exiting Tool run with output:
""
[chain/start] [1:chain:AgentExecutor > 8:chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "I can use the `sorted()` function to sort the list of customers. I will need to provide a key function that specifies the sorting criteria.\nAction: Python REPL\nAction Input: `customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]`\nObservation: \nThought:I will use the `sorted()` function to sort the list of customers. I will provide a lambda function as the key argument to specify the sorting criteria.\nAction: Python REPL\nAction Input: `sorted(customers, key=lambda x: (x[1], x[0]))`\nObservation: \nThought:",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:chain:AgentExecutor > 8:chain:LLMChain > 9:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nIf you get an error, debug your code and try again.\nOnly use the output of your code to answer the question. \nYou might know the answer without running any code, but you should still run the code to get the answer.\nIf it does not seem like you can write code to answer the question, just return \"I don't know\" as the answer.\n\n\nPython REPL: A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Python REPL]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]\nThought:I can use the `sorted()` function to sort the list of customers. I will need to provide a key function that specifies the sorting criteria.\nAction: Python REPL\nAction Input: `customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]`\nObservation: \nThought:I will use the `sorted()` function to sort the list of customers. I will provide a lambda function as the key argument to specify the sorting criteria.\nAction: Python REPL\nAction Input: `sorted(customers, key=lambda x: (x[1], x[0]))`\nObservation: \nThought:"
  ]
}
[llm/end] [1:chain:AgentExecutor > 8:chain:LLMChain > 9:llm:ChatOpenAI] [2.72s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The customers have been sorted by last name and then first name.\nFinal Answer: [['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Dolly', 'Too']]",
        "generation_info": null,
        "message": {
          "content": "The customers have been sorted by last name and then first name.\nFinal Answer: [['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Dolly', 'Too']]",
          "additional_kwargs": {},
          "example": false
        }
      }
    ]
  ],
  "llm_output": {
    "token_usage": {
      "prompt_tokens": 486,
      "completion_tokens": 67,
      "total_tokens": 553
    },
    "model_name": "gpt-3.5-turbo"
  }
}
[chain/end] [1:chain:AgentExecutor > 8:chain:LLMChain] [2.72s] Exiting Chain run with output:
{
  "text": "The customers have been sorted by last name and then first name.\nFinal Answer: [['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Dolly', 'Too']]"
}
[chain/end] [1:chain:AgentExecutor] [8.76s] Exiting Chain run with output:
{
  "output": "[['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Dolly', 'Too']]"
}
```

This debug mode can also be used to highlight what's going wrong, as shown above in the Wikipedia example. Sometimes agents act a little funny and so having all this information is really helpful for understanding what's going on.

## Define your own tool

In [ ]:
#!pip install DateTime

In [ ]:
from langchain.agents import tool
from datetime import date

A big power of agents is that you can connect it to your own sources of information, your own APIs, your own data. So here we're going to go over how you can create a custom tool so that you can connect it to whatever you want.

Let's make a tool that's going to tell us what the current data is. First, we're going to import this tool decorator. This can be applied to any function and it turns it into a tool that Langchang can use.

Next, we're going to write a function called Time which takes in any text string. We're not really going to use that and it's going to return today's date by calling date Time. In addition to the name of the function, we're also going to write a really detailed doc string.

That's because this is what the agent will use to know when it should call this tool and how it should call this tool. For example, here we say that the input should always be an empty string. That's because we don't use it if we have more stringent requirements on what the input should be.

For example, if we have a function that should always take in a search query or a SQL statement, you'll want to make sure to mention that here we're now going to create another agent. This time we're adding the Time tool to the list of existing tools. And finally, let's call the agent and ask it what the date today is.

It recognizes that it needs to use the Time tool which it specifies. Here it has the action input as an empty string. This is great.

This is what we told it to do. And then it returns with an observation. And then finally the language model takes that observation and responds to the user.

In [ ]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [ ]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

*OUTPUT*

![Agent](immagini/33_agent.png)

**Note**: 

The agent will sometimes come to the wrong conclusion (agents are a work in progress!). 

If it does, please try running it again.

In [ ]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")

```python
> Entering new AgentExecutor chain...
Question: What's the date today?
Thought: I can use the `time` tool to get the current date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```
Observation: 2023-08-02
Thought:
Observation: Invalid or incomplete response
Thought:I made a mistake in my previous response. I apologize for the confusion. Let me correct it and provide the final answer.

Question: What's the date today?
Thought: I can use the `time` tool to get the current date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2023-08-02
Thought:I now know the final answer.
Final Answer: The date today is August 2, 2023.

> Finished chain.
```